##Classifying MNIST handwriting digits with Multi Layer Perceptron(MLP)

* In this session, we will create a fully connected MLP with one hidden layer, train and evaluate the network on the MNIST dataset. 
* The MNIST dataset is small enough that we can use a simple MLP for training( (The MNIST images are relatively small)

##What does our planned MLP looks like? Draw the diagram 

* How many inputs?
* How many hidden layers?
* How many neurons in the hidden layers?
* How many neurons in the output layer?
* What about activation functions?


##The Main Steps

Generally, the main steps for building a Deep Learning Neural Network are as follows. 

1. Import libraries, seed
2. Set data preprocessing (transform), download dataset, split train and test 
3. Set Dataloaders 
4. Define the model class
5. Set loss function, optimizer and learning rate
6. Training : Load the data
7. Training : Zero the parameter gradients
8. Training : Compute fwd
9. Training : Compute loss
10. Training : Compute backward,set optimizer(update weights)
10. Evaluation of trained model on test dataset

##1. Import libraries, seeding random for reproducibility

In [7]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np 
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)


##2. Download the MNIST dataset and Data pre-processing

* Each PIL image is converted to a PyTorch tensor using transforms.ToTensor()
* The 28*28 image data is flattened into a vector using lambda which is a customized transformation. Here lambda returns a new view of the input tensor which is the product of the input tensor's dimension. 

* We are also splitting the MNIST dataset into training and test dataset








In [8]:

transformCustom = transforms.Compose([
                                transforms.ToTensor(), #this convert to tensor
                                transforms.Lambda(lambda x:x.view(-1))  #this flatten 28*28 into a 784 vector for each image
])


train = datasets.MNIST(root='.',train=True,transform=transformCustom, download=True)
test = datasets.MNIST(root='.', train=False, transform=transformCustom,download=True)




#3. DataLoaders

* Previously like in the perceptron assignment, we passed data manually. Depends on the implementation, for each iteration sometimes we pass one row of data, or one mini batch of data, or one whole batch of data. And for the case of Stochastic Gradient Descent, we need to randomly shuffle the dataset or randomly pick one sample from the dataset i.e we have to do it manually

* In PyTorch, we can use DataLoader class that automatically pass the batches of data fetched from a Dataset object. We can also set if we want the DataLoader to shuffle the data and the size of each batch of data.

* Each batch is a tuple containing the images in the first element and the labels in the second

In [17]:
#Set DataLoader
batchSize = 128  # Rule of thumb is to set to the power of 2. In this case 2^7
train_loader = DataLoader(train, batch_size=batchSize,shuffle=True)
test_loader = DataLoader(test,batch_size=batchSize, shuffle=False) # no need to shuffle test data


##Exercise

What is the shape of the train data and train label?

How many batches are there? Is the size for each batch the same ? 

In [15]:
count = 0
for xb, yb in train_loader:
  count+=1
  #print(f'batch: {count} batchsize:{xb.shape}')

print(f'There are {count} batches in train_loader')
#How many batches are there ?   #128*468+96=60,000

There are 469 batches in train_loader


#Do the same for test data (test_loader)

In [16]:
count=0
for x, y in test_loader:
  count+=1
 #print(f'Batch: {count}  test data: {x.shape} test label: {y.shape}') #78*128+16=10,000
  
print(f'There are {count} batches in test_loader') 

There are 79 batches in test_loader


#Define the Neural Net model/class (MLP with one hidden layer, fully connected)

* Define our model in a class that extends nn.Module. 
* nn.Module subclasses must do a minimum of one thing: implement the forward method which takes a batch of data and performs the forward-pass. 

* PyTorch's autograd system will take care of computing the gradients of the forward pass for us. In the code below we'll also make use of the constructor of our model to instantiate the hidden and output layers.


* The nn.Module class defines a instance variable called `training` that is set to True when the model is being trained and False when it is being evaluated after being trained. 

* In our model definition we've used a softmax activation function on the output layer to turn the outputs into probability-like values, but have only set this to be enabled when we are not training the model. We've done this because we will use PyTorch's implementation of Cross Entropy Loss (nn.CrossEntropyLoss) during training which **implicitly** adds a softmax before a logarithmic loss.

* In our case the softmax isn't actually necessary for model evaluation if we're only interested in the most likely class; the logits (unscaled log probabilities) provided by the final fully connected layer before the softmax can be used directly as the largest logit will correspond to the most likely class.


In [12]:
class MLP(nn.Module):
  def __init__(self,input_size, hidden_size,num_classes):
    super(MLP,self).__init__()

    self.layer1 = nn.Linear(input_size,hidden_size) 
    self.layer2 = nn.Linear(hidden_size,num_classes)

  def forward(self,x): 

    out = self.layer1(x)
    #out = F.sigmoid(out) 
    out = torch.sigmoid(out)
    out = self.layer2(out)
 
    if not self.training:
      out = F.softmax(out,dim=1)
    return out


#5 Set loss function, optimizer
#6-10 Training




Training and Evaluating the Model
* One of the design decisions of PyTorch is that everything should be explicit so we have full control over our models and the training process. 

* This means that we actually need to write the model training loop by hand, and perform each of the various operations (perform the forward-pass, compute the loss, perform the backward-pass, and update the weights). 

* In the code below we'll fit the model to the data over several epochs using batches of 128 images provided by the DataLoader defined previously. 

* We'll make use of the ADAM optimiser as it broadly tends to work well practically despite its limitations.

In [18]:

model = MLP(784, 784, 10) #input_size,hidden_size,num_classes


loss_fn = nn.CrossEntropyLoss() 
opt = torch.optim.Adam(model.parameters()) #optimizer, optimization strategy-to escape the local minima and to converge quickly
#rule of thumb for optimizer
#1. if you want to keep things simple. use ADAM
#2. if you have time, then use SGD, and tune the learning rate/parameters 
#3. if you are implementing a paper, use the same strategy as what the authors are using 

epochSize =3 #obviously this isn't enough

for epoch in range(epochSize):
  model.train() #training mode
  loss = 0

  for input_batch, target_batch in train_loader:

    opt.zero_grad() #zero the gradients
    predict_batch = model(input_batch) #fwd
    loss_batch = loss_fn(predict_batch,target_batch) #loss
    loss_batch.backward() #backward

    opt.step()#optimize i.e update weights

    loss += loss_batch.item()
    

  print(f'Epoch: {epoch+1}  loss: {loss}')    
  
        
    


Epoch: 1  loss: 223.8967769742012
Epoch: 2  loss: 110.00368839502335
Epoch: 3  loss: 83.59569402784109


#Visualize epoch vs loss/cost and minibatches vs loss/cost


* In the above we printed out the total loss at the end of each epoch. With your own code, plot the loss.



In [ ]:
#plot the loss by epochs, minibatches here

#11. Evaluation of trained model on test data

* Compute the overall accuracy of the test set. Note that you need a call to model.eval() - this sets the model into evaluation mode and supresses non-training things (gradients, and things such as dropout being applied/computed).

In [ ]:

model.eval() # sets the model in evaluation mode 
for input_batch, target_batch in test_loader:
  
  #Your code here 

print('number of test data')
print(f'number of wrongly predicted label ')


print(f'test accuracy')


